In [3]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [4]:
#importing the dataset
with open('finale.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
data

,inputs,tags
0,hello,greeting
1,hi there,greeting
2,greetings,greeting
3,nice to meet you,greeting
4,heyoo,greeting
...,...,...
101,Mtu anaingia vipi Kwa live system,livesystem
102,Hello can I get assistance in the opening of t...,livesystem
103,Hello can I get the technicians to go about ac...,livesystem
104,How do I get to the live System,livesystem


In [5]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [6]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

34
number of unique words :  240
output length:  14


In [7]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
4/4 [==============================] - 2s 15ms/step - loss: 2.6348 - accuracy: 0.0943
Epoch 2/200
4/4 [==============================] - 0s 17ms/step - loss: 2.6217 - accuracy: 0.1038
Epoch 3/200
4/4 [==============================] - 0s 16ms/step - loss: 2.6066 - accuracy: 0.2075
Epoch 4/200
4/4 [==============================] - 0s 16ms/step - loss: 2.5847 - accuracy: 0.1792
Epoch 5/200
4/4 [==============================] - 0s 17ms/step - loss: 2.5595 - accuracy: 0.1698
Epoch 6/200
4/4 [==============================] - 0s 16ms/step - loss: 2.5310 - accuracy: 0.1698
Epoch 7/200
4/4 [==============================] - 0s 16ms/step - loss: 2.5002 - accuracy: 0.1698
Epoch 8/200
4/4 [==============================] - 0s 16ms/step - loss: 2.4705 - accuracy: 0.1698
Epoch 9/200
4/4 [==============================] - 0s 16ms/step - loss: 2.4489 - accuracy: 0.1698
Epoch 10/200
4/4 [==============================] - 0s 16ms/step - loss: 2.4340 - accuracy: 0.1698
Epoch 11/200
4/4 [=

In [ ]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Maya : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    exit